In [1]:
#pip install tqdm boto3 requests regex
#pip install pycuda
#pip install pytorch_pretrained_bert==0.4.0

# Just list which models can used

            'bert-base-uncased'
            'bert-large-uncased'
            'bert-base-cased': 
            'bert-large-cased'
            'bert-base-multilingual-uncased'
            'bert-base-multilingual-cased'
            'bert-base-chinese'

In [2]:
import torch
import pycuda.driver as cuda
cuda.init()
## Get Id of default device

torch.cuda.current_device()
# 0
cuda.Device(0).name() #

'GeForce RTX 2060'

In [3]:
torch.cuda.get_device_name(0)

'GeForce RTX 2060'

In [4]:
# A simple class to know about your cuda devices
import pycuda.driver as cuda
import pycuda.autoinit # Necessary for using its functions
cuda.init() # Necesarry for using its functions

class aboutCudaDevices():
    def __init__(self):
        pass
    
    def num_devices(self):
        """返回 cuda 设备的数量"""
        return cuda.Device.count()
    
    def devices(self):
        """获取所有可用的设备的名称"""
        num = cuda.Device.count()
        print("%d device(s) found:"%num)
        for i in range(num):
            print(cuda.Device(i).name(), "(Id: %d)"%i)
            
    def mem_info(self):
        """获取所有设备的总内存和可用内存"""
        available, total = cuda.mem_get_info()
        print("Available: %.2f GB\nTotal:     %.2f GB"%(available/1e9, total/1e9))
        
    def attributes(self, device_id=0):
        """返回指定 id 的设备的属性信息"""
        return cuda.Device(device_id).get_attributes()
    
    def __repr__(self):
        """输出设备的数量和其id、内存信息"""
        num = cuda.Device.count()
        string = ""
        string += ("%d device(s) found:\n"%num)
        for i in range(num):
            string += ( "    %d) %s (Id: %d)\n"%((i+1),cuda.Device(i).name(),i))
            string += ("          Memory: %.2f GB\n"%(cuda.Device(i).total_memory()/1e9))
        return string

# You can print output just by typing its name (__repr__):
aboutCudaDevices()
# 1 device(s) found:
#    1) Tesla K80 (Id: 0)
#          Memory: 12.00 GB


1 device(s) found:
    1) GeForce RTX 2060 (Id: 0)
          Memory: 6.44 GB

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import sys
import zipfile
import datetime
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification

VOCAB ='./CQA/data/bert-base-chinese-vocab.txt'
MODEL ='./CQA/data/pytorch_model'

In [6]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [7]:
if os.path.isdir("./CQA/data/"):
    TRAIN_CSV_PATH = './CQA/data/train.csv'
    TEST_CSV_PATH = './CQA/data/test.csv'
    TOKENIZED_TRAIN_CSV_PATH = "./CQA/data/tokenized_train2.csv"
else:
    TRAIN_CSV_PATH = './CQA/data/train.csv'
    TEST_CSV_PATH = './CQA/data/test.csv'
    TOKENIZED_TRAIN_CSV_PATH = ""
    
train = pd.read_csv(TRAIN_CSV_PATH)
test = pd.read_csv(TEST_CSV_PATH)
cols = ['titletext',
        'label']
train = train.loc[:, cols]
test = test.loc[:, cols]
train.fillna('UNKNOWN', inplace=True)
test.fillna('UNKNOWN', inplace=True)
train.head(3)

,titletext,label
0,嚴國安於民國106年10月26日20時10分許，在高雄市鳳山區鳳林路之黃埔公園公車站牌處，見...,搶奪
1,艾康淇前因承攬林徐湘楹（已成年）之房屋裝修工程，並因而發生民事紛爭而在本院涉訟，由本院以10...,妨害名譽
2,黃淑英與黃周招君為母女關係，黃淑英明知其母黃周招君欲將其名下所有、如附表所示之房地贈與予黃淑...,偽造文書


In [8]:
from sklearn.model_selection \
    import train_test_split

VALIDATION_RATIO = 0.1

RANDOM_STATE = 9527

train, val= \
    train_test_split(
        train, 
        test_size=VALIDATION_RATIO, 
        random_state=RANDOM_STATE
)

In [9]:
'''
Counter({'BabyMother': 5009,
         'Baseball': 5017,
         'C_Chat': 5007,
         'EAseries': 5009,
         'HatePolitics': 5013,
         'Japan_Travel': 5009,
         'KoreaStar': 5001,
         'Lifeismoney': 5006,
         'LoL': 5007,
         'MobileComm': 5009,
         'NBA': 5003,
         'PC_Shopping': 5001,
         'Stock': 5008,
         'Tech_Job': 5002,
         'Tennis': 5017,
         'car': 5009,
         'creditcard': 5001,
         'e-shopping': 5010,
         'marvel': 5014,
         'movie': 5009})
'''
label_list = [
'傷害',
'侵占',
'公共危險',
'過失傷害',
'妨害自由',
'妨害公務',
'竊盜',
'妨害名譽',
'妨害性自主',
'偽造文書',
'搶奪',
'毀損',
'妨害風化',
'毒品危害防制',
'妨害性自主罪',
'強盜',
'著作權法',
'其他',
]

In [10]:
from run_classifier import *

train_examples = [InputExample('train', row.titletext, None, row.label) for row in train.itertuples()]
val_examples = [InputExample('val', row.titletext, None, row.label) for row in val.itertuples()]
test_examples = [InputExample('test', row.titletext, None, 'None') for row in test.itertuples()]

len(train_examples)

1204

In [11]:
orginal_total = len(train_examples)
train_examples = train_examples[:int(orginal_total*0.8)]

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
n_gpu = torch.cuda.device_count()
print(n_gpu)
gradient_accumulation_steps = 1
train_batch_size = 2#32
eval_batch_size = 2#128
train_batch_size = train_batch_size // gradient_accumulation_steps
output_dir = './CQA/data/output'
bert_model = 'bert-base-chinese'
num_train_epochs = 3
num_train_optimization_steps = int(
            len(train_examples) / train_batch_size / gradient_accumulation_steps) * num_train_epochs
cache_dir = "model"
learning_rate = 5e-5
warmup_proportion = 0.1
max_seq_length = 512
label_list = [
'傷害',
'侵占',
'公共危險',
'過失傷害',
'妨害自由',
'妨害公務',
'竊盜',
'妨害名譽',
'妨害性自主',
'偽造文書',
'搶奪',
'毀損',
'妨害風化',
'毒品危害防制',
'妨害性自主罪',
'強盜',
'著作權法',
'其他',
]

1


In [13]:
print(len(label_list))

18


In [14]:
tokenizer = BertTokenizer.from_pretrained(bert_model)
model = BertForSequenceClassification.from_pretrained(bert_model,
              cache_dir=cache_dir,
              num_labels = len(label_list))

10/03/2020 17:58:06 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese-vocab.txt from cache at C:\Users\user\.pytorch_pretrained_bert\8a0c070123c1f794c42a29c6904beb7c1b8715741e235bee04aca2c7636fc83f.9b42061518a39ca00b8b52059fd2bede8daa613f8a8671500e518a8c29de8c00
10/03/2020 17:58:07 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-chinese.tar.gz from cache at model\42d4a64dda3243ffeca7ec268d5544122e67d9d06b971608796b483925716512.02ac7d664cff08d793eb00d6aac1d04368a1322435e5fe0a27c70b0b3a85327f
10/03/2020 17:58:07 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file model\42d4a64dda3243ffeca7ec268d5544122e67d9d06b971608796b483925716512.02ac7d664cff08d793eb00d6aac1d04368a1322435e5fe0a27c70b0b3a85327f to temp dir C:\Users\user\AppData\Local\Temp\tmpwefhler0
10/03/2020 17:58:10 - INFO - pytorch_pretraine

In [15]:
model.to(device)
if n_gpu > 1:
    model = torch.nn.DataParallel(model)
model, tokenizer

(BertForSequenceClassification(
   (bert): BertModel(
     (embeddings): BertEmbeddings(
       (word_embeddings): Embedding(21128, 768, padding_idx=0)
       (position_embeddings): Embedding(512, 768)
       (token_type_embeddings): Embedding(2, 768)
       (LayerNorm): BertLayerNorm()
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): BertEncoder(
       (layer): ModuleList(
         (0): BertLayer(
           (attention): BertAttention(
             (self): BertSelfAttention(
               (query): Linear(in_features=768, out_features=768, bias=True)
               (key): Linear(in_features=768, out_features=768, bias=True)
               (value): Linear(in_features=768, out_features=768, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): BertSelfOutput(
               (dense): Linear(in_features=768, out_features=768, bias=True)
               (LayerNorm): BertLayerNorm()
               (dropout): Dropout(p=

In [16]:
# Prepare optimizer
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
optimizer = BertAdam(optimizer_grouped_parameters,
                             lr=learning_rate,
                             warmup=warmup_proportion,
                             t_total=num_train_optimization_steps)

In [17]:

global_step = 0
nb_tr_steps = 0
tr_loss = 0

train_features = convert_examples_to_features(
    train_examples, label_list, max_seq_length, tokenizer)
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", train_batch_size)
logger.info("  Num steps = %d", num_train_optimization_steps)
all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)

train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)

model.train()

print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("Optimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])
    
'''
torch.save(model.state_dict(), 'BertModel')

torch.cuda.empty_cache()

model = torch.load("BertModel", map_location=torch.device('cpu'))
'''

10/03/2020 17:58:13 - INFO - run_classifier -   *** Example ***
10/03/2020 17:58:13 - INFO - run_classifier -   guid: train
10/03/2020 17:58:13 - INFO - run_classifier -   tokens: [CLS] 查 被 告 周 柏 宏 因 違 反 毒 品 危 害 防 制 條 例 案 件 ， 於 偵 查 中 經 具 保 人 周 竹 良 依 臺 灣 新 北 地 方 法 院 檢 察 署 檢 察 官 指 定 保 證 金 新 臺 幣 5 萬 元 ， 而 提 出 現 金 具 保 釋 放 在 案 ， 有 臺 灣 新 北 地 方 法 院 檢 察 署 刑 事 保 證 金 收 據 1 紙 在 卷 可 佐 （ 見 臺 灣 新 北 地 方 法 院 檢 察 署 103 年 度 偵 字 第 186 ##26 號 偵 查 卷 第 266 頁 背 面 ） ， 茲 因 被 告 於 本 院 104 年 度 原 訴 字 第 40 號 毒 品 危 害 防 制 條 例 案 件 準 備 程 序 時 ， 經 合 法 傳 喚 ， 並 通 知 具 保 人 督 促 被 告 到 庭 應 訊 ， 被 告 無 正 當 理 由 而 未 遵 期 到 庭 ， 復 經 本 院 拘 提 無 著 ； 又 具 保 人 經 通 知 亦 未 遵 期 通 知 或 帶 同 被 告 到 案 接 受 應 訊 等 情 ， 此 有 本 院 送 達 證 書 3 份 、 拘 提 報 告 、 具 保 人 之 戶 役 政 連 結 作 業 系 統 查 詢 資 料 各 1 份 附 卷 可 憑 （ 見 本 院 卷 第 38 頁 、 第 40 至 41 頁 、 第 98 頁 、 第 52 頁 ） 。 又 被 告 未 受 羈 押 及 在 監 執 行 ， 顯 已 逃 匿 等 情 ， 復 經 本 院 依 職 權 查 明 屬 實 ， 亦 有 臺 灣 高 等 法 院 被 告 在 監 在 押 全 國 紀 錄 表 1 份 附 卷 足 憑 。 揆 諸 前 揭 說 明 ， 自 應 將 具 保 人 原 繳 納 之 上 開 保 證 金 及 實 收 利 息 沒 入 之 。 [SEP]
10/03/2020 17:58:13 - INF

Model's state_dict:
bert.embeddings.word_embeddings.weight 	 torch.Size([21128, 768])
bert.embeddings.position_embeddings.weight 	 torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight 	 torch.Size([2, 768])
bert.embeddings.LayerNorm.weight 	 torch.Size([768])
bert.embeddings.LayerNorm.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight 	 torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight 	 torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight 	 torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight 	 torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight 	 torch.Size([768])
bert.encoder.layer.0.attention.outp

'\ntorch.save(model.state_dict(), \'BertModel\')\n\ntorch.cuda.empty_cache()\n\nmodel = torch.load("BertModel", map_location=torch.device(\'cpu\'))\n'

In [18]:
for _ in trange(int(num_train_epochs), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    total_step = len(train_data) // train_batch_size
    ten_percent_step = total_step // 10
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch
        loss = model(input_ids, segment_ids, input_mask, label_ids)
        if n_gpu > 1:
            loss = loss.mean() # mean() to average on multi-gpu.
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps

        loss.backward()

        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
            
        if step % ten_percent_step == 0:
            print("Fininshed: {:.2f}% ({}/{})".format(step/total_step*100, step, total_step))

Epoch:  67%|██████▋   | 2/3 [05:31<02:43, 163.91s/it]10/03/2020 18:06:38 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
10/03/2020 18:06:39 - WARNING - pytorch_pretrained_bert.optimization -   Training beyond specified 't_total'. Learning rate multiplier set to 0.0. Please set 't_total' of WarmupLinearSchedule correctly.
Epoch: 100%|██████████| 3/3 [08:22<00:00, 167.53s/it]


Fininshed: 0.00% (0/481)
Fininshed: 9.98% (48/481)
Fininshed: 19.96% (96/481)
Fininshed: 29.94% (144/481)
Fininshed: 39.92% (192/481)
Fininshed: 49.90% (240/481)
Fininshed: 59.88% (288/481)
Fininshed: 69.85% (336/481)
Fininshed: 79.83% (384/481)
Fininshed: 89.81% (432/481)
Fininshed: 99.79% (480/481)
Fininshed: 0.00% (0/481)
Fininshed: 9.98% (48/481)
Fininshed: 19.96% (96/481)
Fininshed: 29.94% (144/481)
Fininshed: 39.92% (192/481)
Fininshed: 49.90% (240/481)
Fininshed: 59.88% (288/481)
Fininshed: 69.85% (336/481)
Fininshed: 79.83% (384/481)
Fininshed: 89.81% (432/481)
Fininshed: 99.79% (480/481)
Fininshed: 0.00% (0/481)
Fininshed: 9.98% (48/481)
Fininshed: 19.96% (96/481)
Fininshed: 29.94% (144/481)
Fininshed: 39.92% (192/481)
Fininshed: 49.90% (240/481)
Fininshed: 59.88% (288/481)
Fininshed: 69.85% (336/481)
Fininshed: 79.83% (384/481)
Fininshed: 89.81% (432/481)
Fininshed: 99.79% (480/481)


In [19]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save a trained model and the associated configuration
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
torch.save(model_to_save.state_dict(), output_model_file)
output_config_file = os.path.join(output_dir, CONFIG_NAME)
with open(output_config_file, 'w') as f:
    f.write(model_to_save.config.to_json_string())

In [20]:
# Load a trained model and config that you have fine-tuned
output_config_file = os.path.join(output_dir, CONFIG_NAME)
output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
config = BertConfig(output_config_file)
model = BertForSequenceClassification(config, num_labels=len(label_list))
model.load_state_dict(torch.load(output_model_file))
model.to(device)  # important to specific device
if n_gpu > 1:
    model = torch.nn.DataParallel(model)

In [21]:
config

{
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

In [22]:
# val
eval_examples = val_examples
eval_features = convert_examples_to_features(
    eval_examples, label_list, max_seq_length, tokenizer)
logger.info("***** Running evaluation *****")
logger.info("  Num examples = %d", len(eval_examples))
logger.info("  Batch size = %d", eval_batch_size)
all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for input_ids, input_mask, segment_ids, label_ids in eval_dataloader:
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
        logits = model(input_ids, segment_ids, input_mask)

    logits = logits.detach().cpu().numpy()
    label_ids = label_ids.to('cpu').numpy()
    tmp_eval_accuracy = accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += input_ids.size(0)
    nb_eval_steps += 1

eval_loss = eval_loss / nb_eval_steps
eval_accuracy = eval_accuracy / nb_eval_examples
loss = tr_loss/nb_tr_steps
result = {'eval_loss': eval_loss,
          'eval_accuracy': eval_accuracy,
          'global_step': global_step,
          'loss': loss}

output_eval_file = os.path.join(output_dir, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    logger.info("***** Eval results *****")
    for key in sorted(result.keys()):
        logger.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

10/03/2020 18:06:42 - INFO - run_classifier -   *** Example ***
10/03/2020 18:06:42 - INFO - run_classifier -   guid: val
10/03/2020 18:06:42 - INFO - run_classifier -   tokens: [CLS] 被 告 陳 盈 貝 於 民 國 104 年 2 月 24 日 晚 間 8 時 16 分 許 ， 駕 駛 車 牌 號 碼 000 ##0 - 00 號 之 自 用 小 客 車 ， 沿 桃 園 市 桃 園 區 桃 鶯 路 往 桃 園 區 方 向 行 駛 ， 行 經 桃 園 市 ○○ 區 ○○ 路 000 號 前 ， 本 應 注 意 後 方 車 超 越 前 行 車 時 ， 應 保 持 半 公 尺 以 上 之 間 隔 超 過 ， 且 依 當 時 情 況 並 無 不 能 注 意 之 情 事 ， 竟 疏 未 注 意 ， 於 未 保 持 安 全 距 離 之 情 況 下 ， 超 越 前 方 告 訴 人 李 金 玉 所 騎 乘 車 牌 號 碼 000 - 000 號 之 普 通 重 型 機 車 ， 兩 車 遂 因 此 發 生 碰 撞 ， 致 告 訴 人 因 此 受 有 頸 椎 外 傷 併 椎 間 盤 突 出 、 脊 髓 病 變 之 傷 害 。 [SEP]
10/03/2020 18:06:42 - INFO - run_classifier -   input_ids: 101 6158 1440 7376 4659 6509 3176 3696 1751 8503 2399 123 3299 8125 3189 3241 7279 129 3229 8121 1146 6258 8024 7690 7691 6722 4277 5998 4826 8241 8129 118 8136 5998 722 5632 4500 2207 2145 6722 8024 3784 3425 1754 2356 3425 1754 1281 3425 7873 6662 2518 3425 1754 1281 3175 1403 6121 7691 8024 6121 5195 3425 1754 2356 9423 1281 94

In [23]:
def predict(model, tokenizer, examples, label_list, eval_batch_size=128):
    model.to(device)
    eval_examples = examples
    eval_features = convert_examples_to_features(
        eval_examples, label_list, max_seq_length, tokenizer)
    logger.info("***** Running evaluation *****")
    logger.info("  Num examples = %d", len(eval_examples))
    logger.info("  Batch size = %d", eval_batch_size)
    all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
    eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    # Run prediction for full data
    eval_sampler = SequentialSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    
    res = []
    for input_ids, input_mask, segment_ids, label_ids in eval_dataloader:
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
#         label_ids = label_ids.to(device)

        with torch.no_grad():
#             tmp_eval_loss = model(input_ids, segment_ids, input_mask, label_ids)
            logits = model(input_ids, segment_ids, input_mask)

        logits = logits.detach().cpu().numpy()
#         print(logits)
        res.extend(logits.argmax(-1))
#         label_ids = label_ids.to('cpu').numpy()
#         tmp_eval_accuracy = accuracy(logits, label_ids)

#         eval_loss += tmp_eval_loss.mean().item()
#         eval_accuracy += tmp_eval_accuracy

#         nb_eval_examples += input_ids.size(0)
        nb_eval_steps += 1

#     eval_loss = eval_loss / nb_eval_steps
#     eval_accuracy = eval_accuracy / nb_eval_examples
#     loss = tr_loss/nb_tr_steps 
#     result = {'eval_loss': eval_loss,
#               'eval_accuracy': eval_accuracy,
#               'global_step': global_step,
#               'loss': loss}

#     output_eval_file = os.path.join(output_dir, "eval_results.txt")
#     with open(output_eval_file, "w") as writer:
#         logger.info("***** Eval results *****")
#         for key in sorted(result.keys()):
#             logger.info("  %s = %s", key, str(result[key]))
#             writer.write("%s = %s\n" % (key, str(result[key])))
    return res
    

In [24]:
res = predict(model, tokenizer, test_examples, label_list)

KeyError: 'None'

In [ ]:
label_list

In [ ]:
set(res)

In [ ]:
predict(model, tokenizer, test_examples[:10], label_list)

In [ ]:
cat_map = {idx:lab for idx, lab in enumerate(label_list)}
res = [cat_map[c] for c  in res]

In [ ]:
test.tail()

In [ ]:
#　For Submission

test['label'] = res


submission = test

submission.columns = ['titletext', 'label']
submission.to_csv('./CQA/data/submission.csv', index=False)
submission.head()

In [ ]:
test_r = pd.read_csv(TEST_CSV_PATH)

In [ ]:
test_r.head()

In [ ]:
test_m = pd.merge(test, test_r, left_index=True, right_index=True)
test_m = test_m[["titletext_x", "Category", "label" ]]
test_m.columns = ['titletext', 'Category', 'label']
test_m.to_csv('./CQA/data/submission_compare.csv', index=False)
test_m.head()

In [ ]:
test_m.head()

In [ ]:
test_fail = test_m[test_m["Category"]==test_m["label"]]

In [ ]:
test_fail.count()

In [ ]:
test_fail.count()